In [ ]:
import numpy as np
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_mat import *
from block_rect_maxvol import *
import scipy.linalg
%matplotlib inline

In [ ]:
nder = 2 # 
expansion = 3
n = 600 # number of points
n_test = 5000 #points on test grid
p_size = (nder+1)*expansion #number of monoms (number of columns)
interval = [-1, 1]  # interval of x_test
x = 2*np.random.rand(n, nder)-1
x_test = np.random.rand(n_test,nder)

In [ ]:
c = GenMat(p_size, x,poly=cheb, poly_diff=cheb_diff, debug=False,pow_p=1)
# c = GenMat(p_size, x, poly=trigpoly, poly_diff=trigpoly_diff, debug=False, pow_p=1)
print c.shape
# turn matrix to the block structure
A = matrix_prep(c,nder+1,n)
print A.shape

In [ ]:
### returns 2 values - function on domain, and block structured
def rhs(points, nder, mode = 'gauss'):
    if mode == 'gauss':
        block_rhs = np.zeros((nder+1)*(points.shape[0]))
        func = 2*np.exp(-((points[:,0]**2)/2. + (points[:,1]**2)/2.))# + (points[:,2]**2)/2.)) # Gaussian
        for i in range(points.shape[0]):
            block_rhs[i*(nder+1)] = func[i]
            for j in range(nder):
                block_rhs[i*(nder+1)+j+1] = -1*(points[i,j])*func[i]
    if mode == '3D' :
        block_rhs = np.zeros((nder+1)*(points.shape[0]))
        func = 2*((points[:,0]**2)/2. + (points[:,1]**2)/2. + (points[:,2]**2)/2.) # Gaussian
        for i in range(points.shape[0]):
            block_rhs[i*(nder+1)] = func[i]
            for j in range(nder):
                block_rhs[i*(nder+1)+j+1] = 2 * points[i,j]
    return func, block_rhs     

#### Information about initial matrix

In [ ]:
print la.det(A[:A.shape[1]])
print la.matrix_rank(A)
print la.matrix_rank(A[:A.shape[1]])
_,s,_ = scipy.linalg.svd(A)
print s

piv,_ = maxvol(A)
print piv
print la.det(A[piv])

#### Rectangular block maxvol

In [ ]:
k = 15 ### number of rows for LSM
assert k//(nder+1)!=0
assert (k>=A.shape[1])
a,b,final_piv,d,e = rect_block_maxvol(A, nder, Kmax = k, max_iters=100, rect_tol = 0.05, tol = 0.0, debug = False, ext_debug = True)

In [ ]:
### solve LSM
f,t = rhs(x,nder, mode = 'gauss')
A_test = GenMat(p_size, x_test,poly=cheb, debug=False,pow_p=1, ToGenDiff=False)

In [ ]:
f_test,_ = rhs(x_test,nder)

In [ ]:
### Take number of rows that we really need
c = final_piv[:k]

In [ ]:
c_block, res_x, rank, s = np.linalg.lstsq(A[c],t[c])

In [ ]:
taken_p = x[c[::(nder+1)]/(nder+1),:]

In [ ]:
# visualization part
# position of the chosen points by MaxVol and GD in 2-D case
l_bound = interval[0]
u_bound = interval[1]
plt.xlim(l_bound-0.15, u_bound+0.15)
plt.ylim(l_bound-0.15, u_bound+0.15)
#plt.plot(M[row_indx,1], M[row_indx,2], 'ro', label = "MV")
plt.plot(taken_p[:,0],taken_p[:,1], 'b^', label = "BMV")
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, borderaxespad=0.)
plt.grid(True)
plt.show()

In [ ]:
print("GD error = ", la.norm(f_test - np.dot(A_test, c_block), np.inf) / la.norm(f_test, np.inf), "\n")

In [ ]:
apr_calcul = approximant(nder,c_block)
error_est(gauss,apr_calcul,x_test)

### Testing the new most general environment

In [ ]:
### generating test points
points_test = test_points_gen(n_test,nder,distrib='random') 
### evaluating test
N_rows = 15
error = test(A,x,points_test,nder,expansion,N_rows,gauss)